In [1]:
import race_results_scanner
import google_cloud_downloader as gcd
from PyPDF2 import PdfReader
import numpy as np
import pandas as pd
import os
import header_scanner as hs
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',None)

pdf = "Error_PDFS/BAQ_09_15_2022.pdf"
#pdf = "Example_PDFS/equibaseFile.pdf"


In [2]:
#Testing for page

page_list = race_results_scanner.get_page_list(pdf)
target_page = 1

page_list_index = 0
for page in page_list:
    if(target_page == page['page_num']):
        break
    page_list_index += 1
    
reader = PdfReader(pdf) #File to be scanned
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()

page = page_list[page_list_index]
header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
for field,value in header.items():
    print(field ,':', value)
result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count'],page['last_pgm']) #Table scan

The PDF <_io.BufferedReader name='Error_PDFS/BAQ_09_15_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


track :  BELMONT AT THE BIG A
date :  September 15, 2022
race_num :  2
criteria :  TO BE RUN OVER NATIONAL FENCES A HURDLE HANDICAP FOR FOUR-YEAR-OLDS AND UPWARD.
track_length :  (7)(e)(5) Lasix not permitted within 48 hours of post time. Closed Saturday, September 3, 2022 with 12 Nominations. Two And One Half Miles On The Hurdle
purse :  $150,000
weather :  Clear
track_type :  Firm
off_time :  1 43
start :  Good for all
track : Belmont At the Big A
date : September 15, 2022
race_num : 2
criteria : TO BE RUN OVER NATIONAL FENCES A HURDLE HANDICAP FOR FOUR-YEAR-OLDS AND UPWARD
track_length :  Two And One Half Miles On The Hurdle
purse : $150,000
weather : Clear
track_type : Firm
off_time : 1:43
start : Good for all
race_type : stakes
Looking for tables...
Found table top bound at 134.12199999999996
Found table bottom bound at 230.822
Looking for tables...
Found table top bound at 369.122
Found table bottom bound at 479.567
    Pgm           Horse Name 1/2  Unnamed: 0     1m   11/2     2

In [4]:
result_tables[1]

,Pgm,Horse Name,1/2,1m,11/2,2m,Str,Fin,1m_length_behind,11/2_length_behind,2m_length_behind,Str_length_behind,Fin_length_behind
0,4.0,Noah And The Ark,(IRE)6,6,3,3,1,1,6,1 1/2,2,6 1/2,9
1,1.0,Snap Decision,1,1,1,1,2,2,3,1/2,1 1/2,6 1/2,9
2,5.0,Ask Paddington (IRE),7,7,7,5,3,3,7 1/2,6 1/2,5 1/2,8,9 3/4
3,6.0,Song for Someone,3,2,2,2,4,4,3,1/2,1 1/2,8 1/2,9 3/4
4,8.0,Amschel (GB),5,3,4,4,5,5,NaN,NaN,NaN,NaN,NaN
5,7.0,Redicean (GB),4,5,8,8,6,6,4,3 1/2,4 1/2,14 1/2,18 3/
6,3.0,Belfast Banter (IRE),8,8,5,6,8,7,5 1/2,7,7 1/2,18 1/2,25
7,2.0,Iranistan,2,4,6,7,7,8,9,5 1/2,6 1/2,49,60 3/


In [6]:
top_table = result_tables[0]
bottom_table = result_tables[1]

bottom_table = bottom_table.drop("Horse Name", axis = 1)
top_table = top_table.astype({'Pgm' : float})
bottom_table = bottom_table.astype({'Pgm' : float})
merged_df = top_table.join(bottom_table.set_index("Pgm"), on = "Pgm", rsuffix = "_RLP")
merged_df

,Last Raced,Pgm,jockey_first_name,jockey_last_name,PP,1/2,1m,11/2,2m,Str,Fin,Odds,Comments,horse_name,Last Raced_super_script,1m_length_behind,11/2_length_behind,2m_length_behind,Str_length_behind,Fin_length_behind,Wgt,M/E,1/2_RLP,1m_RLP,11/2_RLP,2m_RLP,Str_RLP,Fin_RLP,1m_length_behind_RLP,11/2_length_behind_RLP,2m_length_behind_RLP,Str_length_behind_RLP,Fin_length_behind_RLP
0,31Aug22 SAR,2.0,Jamie,Bargary,2.0,6,6,5,5,2,1,8.50,"jumped well, up",Proven Innocent,1 1,NaN,Head,4,5,Head,149,- -,6,6,5,5,2,1,5,1 3/4,1 3/4,1,Hea
1,16Oct21 FH,1.0,Parker,Hendriks,1.0,3,4,1,1,1,2,0.35*,"3-4w pursuit, na",The Mean Queen,3 1,1/2,Head,Head,1,6 3/4,158,h,3,4,1,1,1,2,2 1/2,Head,Head,1,Hea
2,3Aug22 SAR,6.0,Barry,Foley,5.0,5,5,3,4,3,3,4.50,"2-3w pursuit, ke",Howyabud,1 1,2,1/2,1,2,5 1/4,156,- -,5,5,3,4,3,3,3,1,3/4,6,6 3/4
3,17Aug22 SAR,4.0,Danny,Mullins,4.0,2,2,4,6,5,4,10.50,"late, bobbled 7",Going Country,1 2,Head,Head,NaN,2,3/4,153,v,2,2,4,6,5,4,1 1/2,1 1/2,5 3/4,11 1/2,12
4,21May22 MAL,3.0,Sam,Twiston-Davies,3.0,4,3,6,2,4,5,33.25,"tracked inside, ti",Booby Trap,4 6,1,NaN,Head,3 1/2,3 3/4,153,- -,4,3,6,2,4,5,1 1/2,1 3/4,Head,8,12 3/
5,10Aug22 SAR,7.0,Thomas,Garner,6.0,1,1,2,3,6,6,18.40,"in hand 2p, tired",State of Affair,1 1,1 1/2,1,1/2,NaN,NaN,153,- -,1,1,2,3,6,6,1 1/2,Head,Head,13 1/2,16 1/


In [5]:
#Testing for whole pdf
master_df = pd.DataFrame()
page_list = race_results_scanner.get_page_list(pdf)
print(page_list)
    
reader = PdfReader(pdf) #File to be scanned
number_of_pages = len(reader.pages) #Number of pages
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()
i = 0
for page in page_list:
    header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
    print(header)
    result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count'],page['last_pgm']) #Table scan
                
    #Combine into page DF
    top_table = result_tables[0].astype(object)
    bottom_table = result_tables[1].astype(object)
    #bottom_table = bottom_table.drop("Horse Name", axis = 1)
    top_table = top_table.astype({'Pgm' : float})
    bottom_table = bottom_table.astype({'Pgm' : float})
    merged_df = top_table.join(bottom_table.set_index("Pgm"), on = "Pgm", rsuffix = "_RLP")
    for field,value in header.items():
        merged_df[field] = value
        #Changing all cols to object type 
        merged_df = merged_df.astype(object)
        #For every page df
    #Merge into master pdf df
    if(len(pdf_df) < 1):
        pdf_df = merged_df
    else:
        pdf_df = pd.merge(pdf_df, merged_df, how = 'outer')
            
    print(pdf_df)
    #Merging into master 
    #if(len(master_df) < 1):
     #   master_df = pdf_df
    #else:
    #    master_df = pd.merge(master_df,pdf_df, how = 'outer')

The PDF <_io.BufferedReader name='Error_PDFS/BAQ_09_15_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


[{'page_num': 0, 'horse_count': 6, 'last_pgm': '7'}, {'page_num': 1, 'horse_count': 8, 'last_pgm': '2'}, {'page_num': 2, 'horse_count': 6, 'last_pgm': '3'}, {'page_num': 3, 'horse_count': 8, 'last_pgm': '6'}, {'page_num': 5, 'horse_count': 6, 'last_pgm': '7'}, {'page_num': 6, 'horse_count': 12, 'last_pgm': '2'}, {'page_num': 8, 'horse_count': 10, 'last_pgm': '7'}, {'page_num': 10, 'horse_count': 6, 'last_pgm': '7'}, {'page_num': 11, 'horse_count': 9, 'last_pgm': '8'}, {'page_num': 13, 'horse_count': 10, 'last_pgm': '9'}]
track :  BELMONT AT /THE BIG A
date :  September 15, 2022
race_num :  1
criteria :  TO BE RUN OVER NATIONAL FENCES A HURDLE FOR FOUR-YEAR OLDS AND UPWARD WHICH HAVE NOT WON OVER HURDLES PRIOR TO MARCH 1, 2021 OR WHICH HAVE NEVER WON THREE RACES, OTHER THAN THREE-YEAR OLDS.
track_length :  (7)(e)(5) Lasix not permitted within 48 hours of post time. Four-year-olds 146 lbs.; Older 150 lbs. Winners, other than maiden, claiming, starter or three-year-olds 3 lbs. extra for e

The PDF <_io.BufferedReader name='Error_PDFS/BAQ_09_15_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table bottom bound at 453.64900000000006
    Pgm            Horse Name    1/2     1m   11/2     2m     Str    Fin
0   NaN                   NaN  4 1/4      5  1 3/4  1 3/4       1    Hea
1   2.0       Proven Innocent      6      6      5      5       2      1
2   NaN                   NaN  1 1/2  2 1/2   Head   Head       1    Hea
3   1.0  The Mean Queen (IRE)      3      4      1      1       1      2
4   NaN                   NaN  2 1/4      3      1    3/4       6  6 3/4
5   6.0        Howyabud (IRE)      5      5      3      4       3      3
6   NaN                   NaN   Head  1 1/2  1 1/2  5 3/4  11 1/2     12
7   4.0   Going Country (IRE)      2      2      4      6       5      4
8   NaN                   NaN  1 3/4  1 1/2  1 3/4   Head       8  12 3/
9   3.0            Booby Trap      4      3      6      2       4      5
10  NaN                   NaN   Head  1 1/2   Head   Head  13 1/2  16 1/
11  7.0       State of Affair      1      1      2      3       6      6
    

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Error_PDFS/BAQ_09_15_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


...

track :  BELMONT AT THE BIG *A
date :  September 15, 2022
race_num :  9
criteria :  FOR FILLIES AND MARES THREE YEARS OLD AND UPWARD FOALED IN NEW YORK STATE AND APPROVED BY THE NEW YORK STATE-BRED REGISTRY WHICH HAVE NEVER WON $18,000 OTHER THAN MAIDEN, CLAIMING OR STARTER OR WHICH HAVE NEVER WON TWO RACES.
track_length :  lbs.; Older, 124 lbs. Non-winners Of A Race Other Than Claiming Or Starter At A Mile Or Over Allowed 2 lbs. (SNW1$ X) One Mile On The Dirt
purse :  $85,000
weather :  Clear
track_type :  Fast
off_time :  5 20
start :  Good for all except 3,6
{'track': 'Belmont At the Big A', 'date': 'September 15, 2022', 'race_num': '9', 'criteria': 'FOR FILLIES AND MARES THREE YEARS OLD AND UPWARD FOALED IN NEW YORK STATE AND APPROVED BY THE NEW YORK STATE-BRED REGISTRY WHICH HAVE NEVER WON $18,000 OTHER THAN MAIDEN, CLAIMING OR STARTER OR WHICH HAVE NEVER WON TWO RACES', 'track_length': ' One Mile On The Dirt', 'purse': '$85,000', 'weather': 'Clear', 'track_type': 'Fast', 'off_tim

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Error_PDFS/BAQ_09_15_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table bottom bound at 531.901
    Pgm        Horse Name  Start    1/4     1/2     3/4     Str    Fin
0   NaN               NaN    NaN    1/2     1/2     1/2   2 1/2      3
1   5.0           Truancy    4.0      2       2       1       1      1
2   NaN               NaN    NaN  2 3/4   2 1/2       1   2 1/2      3
3   1.0     Sweet Mystery    7.0      6       4       3       2      2
4   NaN               NaN    NaN  2 1/2       3       5   7 1/2     11
5   6.0         Caramocha    6.0      5       5       4       4      3
6   NaN               NaN    NaN      2   3 1/2       7   9 1/2     11
7   9.0         Gringotts    2.0      4       6       5       5      4
8   NaN               NaN    NaN    1/2   2 1/2     1/2       6  16 1/
9   7.0  Captainsdaughter    3.0      3       3       2       3      5
10  NaN               NaN    NaN  7 3/4       9       9      11  16 1/
11  2.0      Mia Bea Star    9.0      9       8       6       6      6
12  NaN               NaN    NaN  6 3/4  

Looking for tables...
Found table top bound at 141.7059999999999
Found table bottom bound at 253.00600000000003
Looking for tables...
Found table top bound at 514.601
Found table bottom bound at 629.2349999999999
     Pgm          Horse Name  Start    1/4    1/2     Str     Fin
0    NaN                 NaN    NaN  1 1/2      2   2 1/2       2
1    5.0        Rugged Union    1.0      1      1       1       1
2    NaN                 NaN    NaN      3      2   2 1/2       2
3    8.0  Theregoesmymiracle    3.0      3      2       2       2
4    NaN                 NaN    NaN  4 1/2      4       5       5
5    7.0        Empire Ridge    4.0      5      4       3       3
6    NaN                 NaN    NaN  5 1/2  5 1/2       7       7
7    4.0    Impazible Prince    2.0      7      6       4       4
8    NaN                 NaN    NaN  7 1/2  7 1/2   9 3/4       8
9    2.0   Southern Civility    7.0      9      9       9       5
10   NaN                 NaN    NaN      5      6   9 1/2   8

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [6]:
pdf_df

,Last Raced,Pgm,jockey_first_name,jockey_last_name,PP,1/2,1m,11/2,2m,Str,Fin,Odds,Comments,horse_name,Last Raced_super_script,1m_length_behind,11/2_length_behind,2m_length_behind,Str_length_behind,Fin_length_behind,Wgt,M/E,Horse Name,1/2_RLP,1m_RLP,11/2_RLP,2m_RLP,Str_RLP,Fin_RLP,1m_length_behind_RLP,11/2_length_behind_RLP,2m_length_behind_RLP,Str_length_behind_RLP,Fin_length_behind_RLP,track,date,race_num,criteria,track_length,purse,weather,track_type,off_time,start,race_type,Start,1/4,1/4_length_behind,1/2_length_behind,Start_RLP,1/4_RLP,1/4_length_behind_RLP,1/2_length_behind_RLP,3/4,3/4_length_behind,3/4_RLP,3/4_length_behind_RLP,claiming_price
0,31Aug22 SAR,2.0,Jamie,Bargary,2.0,6,6,5,5,2,1,8.50,"jumped well, up",Proven Innocent,1 1,NaN,Head,4,5,Head,149,- -,Proven Innocent,6,6,5,5,2,1,5,1 3/4,1 3/4,1,Hea,Belmont At the Big A,"September 15, 2022",1,TO BE RUN OVER NATIONAL FENCES A HURDLE FOR FO...,Two And Three Eighth Miles On The Hurdle,"$75,000",Clear,Firm,1:04,Good for all,stakes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16Oct21 FH,1.0,Parker,Hendriks,1.0,3,4,1,1,1,2,0.35*,"3-4w pursuit, na",The Mean Queen,3 1,1/2,Head,Head,1,6 3/4,158,h,The Mean Queen (IRE),3,4,1,1,1,2,2 1/2,Head,Head,1,Hea,Belmont At the Big A,"September 15, 2022",1,TO BE RUN OVER NATIONAL FENCES A HURDLE FOR FO...,Two And Three Eighth Miles On The Hurdle,"$75,000",Clear,Firm,1:04,Good for all,stakes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3Aug22 SAR,6.0,Barry,Foley,5.0,5,5,3,4,3,3,4.50,"2-3w pursuit, ke",Howyabud,1 1,2,1/2,1,2,5 1/4,156,- -,Howyabud (IRE),5,5,3,4,3,3,3,1,3/4,6,6 3/4,Belmont At the Big A,"September 15, 2022",1,TO BE RUN OVER NATIONAL FENCES A HURDLE FOR FO...,Two And Three Eighth Miles On The Hurdle,"$75,000",Clear,Firm,1:04,Good for all,stakes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17Aug22 SAR,4.0,Danny,Mullins,4.0,2,2,4,6,5,4,10.50,"late, bobbled 7",Going Country,1 2,Head,Head,NaN,2,3/4,153,v,Going Country (IRE),2,2,4,6,5,4,1 1/2,1 1/2,5 3/4,11 1/2,12,Belmont At the Big A,"September 15, 2022",1,TO BE RUN OVER NATIONAL FENCES A HURDLE FOR FO...,Two And Three Eighth Miles On The Hurdle,"$75,000",Clear,Firm,1:04,Good for all,stakes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21May22 MAL,3.0,Sam,Twiston-Davies,3.0,4,3,6,2,4,5,33.25,"tracked inside, ti",Booby Trap,4 6,1,NaN,Head,3 1/2,3 3/4,153,- -,Booby Trap,4,3,6,2,4,5,1 1/2,1 3/4,Head,8,12 3/,Belmont At the Big A,"September 15, 2022",1,TO BE RUN OVER NATIONAL FENCES A HURDLE FOR FO...,Two And Three Eighth Miles On The Hurdle,"$75,000",Clear,Firm,1:04,Good for all,stakes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10Aug22 SAR,7.0,Thomas,Garner,6.0,1,1,2,3,6,6,18.40,"in hand 2p, tired",State of Affair,1 1,1 1/2,1,1/2,NaN,NaN,153,- -,State of Affair,1,1,2,3,6,6,1 1/2,Head,Head,13 1/2,16 1/,Belmont At the Big A,"September 15, 2022",1,TO BE RUN OVER NATIONAL FENCES A HURDLE FOR FO...,Two And Three Eighth Miles On The Hurdle,"$75,000",Clear,Firm,1:04,Good for all,stakes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,22May21 CAM,4.0,Harrison,Beswick,4.0,6,6,3,3,1,1,41.25,"jumped well,wid",Noah And The Ark,5 1,1 1/2,2,2 1/2,6 1/2,9,140,- -,Noah And The Ark,(IRE)6,6,3,3,1,1,6,1 1/2,2,6 1/2,9,Belmont At the Big A,"September 15, 2022",2,TO BE RUN OVER NATIONAL FENCES A HURDLE HANDIC...,Two And One Half Miles On The Hurdle,"$150,000",Clear,Firm,1:43,Good for all,stakes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,17Aug22 SAR,1.0,Graham,Watters,1.0,1,1,1,1,2,2,0.30*,"jumped well,no",Snap Decision,1 1,3,1/2,1 1/2,1 1/2,3/4,168,- -,Snap Decision,1,1,1,1,2,2,3,1/2,1 1/2,6 1/2,9,Belmont At the Big A,"September 15, 2022",2,TO BE RUN OVER NATIONAL FENCES A HURDLE HANDIC...,Two And One Half Miles On The Hurdle,"$150,000",Clear,Firm,1:43,Good for all,stakes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,16Jul22 MR,5.0,Danny,Mullins,5.0,7,7,7,5,3,3,7.70,struggled fence,Ask Paddington,1,1 1/2,1/2,1,1/2,Nose,146,h,Ask Paddin

In [6]:
pdf_df.to_csv("test.csv")